In [95]:
#[P, P, C, 1B, 2B, 3B, SS, OF, OF, OF]
#maximize average points
using JuMP, Cbc, NamedArrays

raw = readcsv("output.csv");

names = raw[:,1];
salaries = raw[:,2];
position = raw[:, 3];
projectedPoints = raw[:,4];
averagePoints = raw[:,5];
playerTeam = raw[:,6];
opponentTeam = raw[:,7];
n = length(names)

# total salary
salary_cap = 50000

#number of players at each position
no_pitchers = 2
no_catchers = 1
no_first = 1
no_second = 1
no_third = 1
no_ss = 1
no_of = 3
total_players = 10

println(n)
println(playerTeam)
println(opponentTeam)

147
Any[" NYM", " WAS", " PHI", " NYY", " LAD", " SEA", " BAL", " ARI", " ATL", " PIT", " HOU", " LAA", " LAA", " CIN", " NYY", " PHI", " WAS", " ARI", " MIL", " NYY", " SEA", " BAL", " HOU", " NYY", " MIL", " HOU", " CIN", " HOU", " NYY", " ARI", " CIN", " LAA", " WAS", " SEA", " SEA", " ATL", " MIL", " LAD", " MIL", " OAK", " SEA", " ATL", " NYY", " OAK", " MIA", " MIA", " PHI", " ARI", " OAK", " SEA", " NYM", " PHI", " PHI", " PHI", " LAD", " LAD", " PIT", " OAK", " SEA", " NYM", " NYM", " WAS", " HOU", " OAK", " OAK", " LAA", " LAD", " CIN", " CIN", " NYY", " HOU", " NYY", " OAK", " ATL", " PHI", " OAK", " SEA", " SEA", " BAL", " BAL", " NYM", " WAS", " CIN", " NYY", " BAL", " LAA", " LAA", " ATL", " PIT", " CIN", " NYY", " LAA", " ARI", " PIT", " WAS", " MIL", " MIL", " HOU", " OAK", " LAA", " PIT", " MIL", " HOU", " ATL", " MIA", " MIA", " MIA", " LAD", " ARI", " HOU", " LAD", " ARI", " PIT", " WAS", " PIT", " OAK", " NYM", " MIA", " PHI", " LAD", " PIT", " CIN", " LAA", " BAL", 

In [96]:
# could store indexes of all pitchers (or other players) and force (constrain) the sum of binaries at those indexes to equal 2
# or different value for however many players are at that position
#array to store indexes of pitchers, catchers, et c.

pitchers = Int64[]
catchers = Int64[]
first = Int64[]
second = Int64[]
third = Int64[]
shortStop = Int64[]
fielders = Int64[]

for i in 1:n
 if contains(position[i], "P")
  append!(pitchers, i)
 end
 if contains(position[i], "C")
  append!(catchers, i)
 end
 if contains(position[i], "1B")
  append!(first, i)
 end
 if contains(position[i], "2B")
  append!(second, i)
 end
 if contains(position[i], "3B")
  append!(third, i)
 end
 if contains(position[i], "SS")
  append!(shortStop, i)
 end
 if contains(position[i], "OF")
  append!(fielders, i)
 end
end

In [97]:
# NL East
phillies = Int64[]
atlanta = Int64[]
miami = Int64[]
washington = Int64[]
mets = Int64[]

# AL East
yankees = Int64[]
baltimore = Int64[]
tampa = Int64[]
boston = Int64[]
toronto = Int64[]

# NL Central
milwaukee = Int64[]
pittsburg = Int64[]
reds = Int64[]
cardinals = Int64[]
chicago = Int64[]

# AL Central
whitesox = Int64[]
royals = Int64[]
tigers = Int64[]
cleveland = Int64[]
twins = Int64[]

# AL West
seattle = Int64[]
angels = Int64[]
oakland = Int64[]
houston = Int64[]
texas = Int64[]

# NL West
dodgers = Int64[]
arizona = Int64[]
padres = Int64[]
giants = Int64[]
rockies = Int64[]

0-element Array{Int64,1}

In [98]:
# NL East
for i in 1:n
 if contains(playerTeam[i], " PHI")
  append!(phillies, i)
 end
 if contains(playerTeam[i], " ATL")
  append!(atlanta, i)
 end
 if contains(playerTeam[i], " MIA")
  append!(miami, i)
 end
 if contains(playerTeam[i], " WAS")
  append!(washington, i)
 end
 if contains(playerTeam[i], " NYM")
  append!(mets, i)
 end
    
# AL East
 if contains(playerTeam[i], " NYY")
  append!(yankees, i)
 end
 if contains(playerTeam[i], " BAL")
  append!(baltimore, i)
 end
 if contains(playerTeam[i], " TB")
  append!(tampa, i)
 end
 if contains(playerTeam[i], " BOS")
  append!(boston, i)
 end
 if contains(playerTeam[i], " TOR")
  append!(toronto, i)
 end
    
 # NL Central
 if contains(playerTeam[i], " MIL")
  append!(milwaukee, i)
 end
 if contains(playerTeam[i], " PIT")
  append!(pittsburg, i)
 end
 if contains(playerTeam[i], " CIN")
  append!(reds, i)
 end
 if contains(playerTeam[i], " STL")
  append!(cardinals, i)
 end
 if contains(playerTeam[i], " CHC")
  append!(chicago, i)
 end
    
# AL Central
  if contains(playerTeam[i], " CWS")
  append!(whitesox, i)
 end
 if contains(playerTeam[i], " KCR")
  append!(royals, i)
 end
 if contains(playerTeam[i], " CLE")
  append!(cleveland, i)
 end
 if contains(playerTeam[i], " DET")
  append!(detroit, i)
 end
 if contains(playerTeam[i], " MIN")
  append!(twins, i)
 end

 if contains(playerTeam[i], " SEA")
  append!(seattle, i)
 end
 if contains(playerTeam[i], " LAA")
  append!(angels, i)
 end
 if contains(playerTeam[i], " OAK")
  append!(oakland, i)
 end
 if contains(playerTeam[i], " HOU")
  append!(houston, i)
 end
 if contains(playerTeam[i], " TEX")
  append!(texas, i)
 end
    
 if contains(playerTeam[i], " LAD")
  append!(dodgers, i)
 end
 if contains(playerTeam[i], " ARI")
  append!(arizona, i)
 end
 if contains(playerTeam[i], " SDP")
  append!(padres, i)
 end
 if contains(playerTeam[i], " SFG")
  append!(giants, i)
 end
 if contains(playerTeam[i], " COL")
  append!(rockies, i)
 end

end

In [99]:
# NL East
oppphillies = Int64[]
oppatlanta = Int64[]
oppmiami = Int64[]
oppwashington = Int64[]
oppmets = Int64[]

# AL East
oppyankees = Int64[]
oppbaltimore = Int64[]
opptampa = Int64[]
oppboston = Int64[]
opptoronto = Int64[]

# NL Central
oppmilwaukee = Int64[]
opppittsburg = Int64[]
oppreds = Int64[]
oppcardinals = Int64[]
oppchicago = Int64[]

# AL Central
oppwhitesox = Int64[]
opproyals = Int64[]
opptigers = Int64[]
oppcleveland = Int64[]
opptwins = Int64[]

# AL West
oppseattle = Int64[]
oppangels = Int64[]
oppoakland = Int64[]
opphouston = Int64[]
opptexas = Int64[]

# NL West
oppdodgers = Int64[]
opparizona = Int64[]
opppadres = Int64[]
oppgiants = Int64[]
opprockies = Int64[]

0-element Array{Int64,1}

In [102]:
# NL East
for i in 1:n
 if contains(opponentTeam[i], " PHI")
  append!(oppphillies, i)
 end
 if contains(opponentTeam[i], " ATL")
  append!(oppatlanta, i)
 end
 if contains(opponentTeam[i], " MIA")
  append!(oppmiami, i)
 end
 if contains(opponentTeam[i], " WAS")
  append!(oppwashington, i)
 end
 if contains(opponentTeam[i], " NYM")
  append!(oppmets, i)
 end
    
# AL East
 if contains(opponentTeam[i], " NYY")
  append!(oppyankees, i)
 end
 if contains(opponentTeam[i], " BAL")
  append!(oppbaltimore, i)
 end
 if contains(opponentTeam[i], " TB")
  append!(opptampa, i)
 end
 if contains(opponentTeam[i], " BOS")
  append!(oppboston, i)
 end
 if contains(opponentTeam[i], " TOR")
  append!(opptoronto, i)
 end
    
 # NL Central
 if contains(opponentTeam[i], " MIL")
  append!(oppmilwaukee, i)
 end
 if contains(opponentTeam[i], " PIT")
  append!(opppittsburg, i)
 end
 if contains(opponentTeam[i], " CIN")
  append!(oppreds, i)
 end
 if contains(opponentTeam[i], " STL")
  append!(oppcardinals, i)
 end
 if contains(opponentTeam[i], " CHC")
  append!(oppchicago, i)
 end
    
# AL Central
  if contains(opponentTeam[i], " CWS")
  append!(oppwhitesox, i)
 end
 if contains(opponentTeam[i], " KCR")
  append!(opproyals, i)
 end
 if contains(opponentTeam[i], " CLE")
  append!(oppcleveland, i)
 end
 if contains(opponentTeam[i], " DET")
  append!(oppdtigers, i)
 end
 if contains(opponentTeam[i], " MIN")
  append!(opptwins, i)
 end

 if contains(opponentTeam[i], " SEA")
  append!(oppseattle, i)
 end
 if contains(opponentTeam[i], " LAA")
  append!(oppangels, i)
 end
 if contains(opponentTeam[i], " OAK")
  append!(oppoakland, i)
 end
 if contains(opponentTeam[i], " HOU")
  append!(opphouston, i)
 end
 if contains(opponentTeam[i], " TEX")
  append!(opptexas, i)
 end
    
 if contains(opponentTeam[i], " LAD")
  append!(oppdodgers, i)
 end
 if contains(opponentTeam[i], " ARI")
  append!(opparizona, i)
 end
 if contains(opponentTeam[i], " SDP")
  append!(opppadres, i)
 end
 if contains(opponentTeam[i], " SFG")
  append!(oppgiants, i)
 end
 if contains(opponentTeam[i], " COL")
  append!(opprockies, i)
 end
end

In [113]:
# no batter vs pitcher constraint
team_abbv = [" PHI", " ATL", " MIA", " WAS", " NYM",
" NYY", " BAL", " TB", " BOS", " TOR", 
" MIL", " PIT", " CIN", " STL", " CHC", 
" CWS", " KCR", " CLE"," DET"," MIN", 
" SEA", " LAA"," OAK"," HOU"," TEX",
" LAD", " ARI", " SDP", " SFG", " COL"]

test = [" PHI"]

noplaylist = Dict(zip(team_abbv, [oppphillies, oppatlanta, oppmiami, oppwashington, oppmets,
            oppyankees, oppbaltimore, opptampa, oppboston, opptoronto,
            oppmilwaukee, opppittsburg, oppreds, oppcardinals, oppchicago,
            oppwhitesox, opproyals, oppcleveland, opptigers, opptwins,
            oppseattle, oppangels, oppoakland, opphouston, opptexas,
            oppdodgers, opparizona, opppadres, oppgiants, opprockies]))

Dict{String,Array{Int64,1}} with 30 entries:
  " CWS" => Int64[]
  " SEA" => [40, 44, 49, 58, 64, 65, 73, 76, 99, 116, 40, 44, 49, 58, 64, 65, 7…
  " CIN" => [19, 25, 37, 39, 96, 97, 102, 127, 130, 19, 25, 37, 39, 96, 97, 102…
  " BOS" => Int64[]
  " PHI" => [45, 46, 105, 106, 107, 118, 126, 139, 147, 45, 46, 105, 106, 107, …
  " DET" => Int64[]
  " MIL" => [14, 27, 31, 68, 69, 83, 90, 122, 136, 14, 27, 31, 68, 69, 83, 90, …
  " OAK" => [6, 21, 34, 35, 41, 50, 59, 77, 78, 141, 6, 21, 34, 35, 41, 50, 59,…
  " ARI" => [5, 38, 55, 56, 67, 108, 111, 120, 5, 38, 55, 56, 67, 108, 111, 120]
  " MIA" => [3, 16, 47, 52, 53, 54, 75, 119, 146, 3, 16, 47, 52, 53, 54, 75, 11…
  " CHC" => Int64[]
  " TB"  => Int64[]
  " TOR" => Int64[]
  " SDP" => Int64[]
  " NYM" => [9, 36, 42, 74, 88, 104, 133, 138, 9, 36, 42, 74, 88, 104, 133, 138]
  " BAL" => [12, 13, 32, 66, 86, 87, 92, 100, 123, 142, 12, 13, 32, 66, 86, 87,…
  " STL" => Int64[]
  " LAA" => [7, 22, 79, 80, 85, 124, 131, 132, 137, 145, 7, 22, 79

In [117]:
m = Model(solver = CbcSolver())

#binary variable for whether a player is chosen or not
@variable(m, x[1:n], Bin)

#constraints must fit line up, eg. two pitchers, one catcher, one 1B etc
@constraint(m, sum(x[i] for i in pitchers) == no_pitchers)
@constraint(m, sum(x[i] for i in catchers) == no_catchers)
@constraint(m, sum(x[i] for i in first) == no_first)
@constraint(m, sum(x[i] for i in second) == no_second)
@constraint(m, sum(x[i] for i in third) == no_third)
@constraint(m, sum(x[i] for i in shortStop) == no_ss)
@constraint(m, sum(x[i] for i in fielders) == no_of)
@constraint(m, sum(x[i] for i in 1:n) == total_players)

# Stacking constraints
@constraint(m, 5 <= sum(x[i] for i in phillies) <= 6)
@constraint(m, 3 <= sum(x[i] for i in milwaukee) <= 5)

# no batter vs pitcher constraint
for i = 1:length(pitchers)
    for j in noplaylist[playerTeam[i]]
      @constraint(m, (x[i] + x[j]) == 1)
    end
end



#total salary must be under 50K
@constraint(m, sum(x .* salaries) <= salary_cap)

#objective to maximize projected points
@objective(m, Max, sum(x .* projectedPoints))

solve(m)

x = getvalue(x)

for i in 1:length(x)
 if x[i] == 1
  println(names[i] , " " , position[i], " ", salaries[i])
 end
end

println(getobjectivevalue(m))

NaN
